# New Testament Expert using Hugging Face Models

This notebook demonstrates how to create a New Testament expert using Hugging Face transformer models specialized in biblical knowledge.

In [ ]:
# Install required packages
!pip install transformers torch accelerate gradio

In [ ]:
from transformers import pipeline, AutoTokenizer, AutoModelForCausalLM
import torch
import gradio as gr
import threading
import time

## Method 1: Using Text Generation Pipeline

In [ ]:
# Use a pipeline as a high-level helper for New Testament questions
nt_pipe = pipeline(
    "text-generation", 
    model="sleepdeprived3/Christian-Bible-Expert-v2.0-12B",
    torch_dtype=torch.float16,
    device_map="auto"
)

# New Testament specific questions
nt_questions = [
    "Who were the twelve apostles of Jesus?",
    "What is the significance of the Sermon on the Mount?",
    "Explain the parable of the Good Samaritan",
    "What happened during the Last Supper?",
    "Describe the resurrection of Jesus Christ"
]

for question in nt_questions:
    print(f"\n🔹 Question: {question}")
    result = nt_pipe(
        f"New Testament Question: {question}\nAnswer:",
        max_length=200,
        num_return_sequences=1,
        temperature=0.7,
        do_sample=True,
        pad_token_id=nt_pipe.tokenizer.eos_token_id
    )
    print(f"📖 Answer: {result[0]['generated_text'].split('Answer:')[1].strip()}")
    print("-" * 80)

## Method 2: Interactive New Testament Expert

In [ ]:
class NewTestamentExpert:
    def __init__(self, model_name="sleepdeprived3/Christian-Bible-Expert-v2.0-12B"):
        print("Loading New Testament Expert model...")
        self.tokenizer = AutoTokenizer.from_pretrained(model_name)
        self.model = AutoModelForCausalLM.from_pretrained(
            model_name,
            torch_dtype=torch.float16,
            device_map="auto",
            trust_remote_code=True
        )
        
        # Set pad token if not available
        if self.tokenizer.pad_token is None:
            self.tokenizer.pad_token = self.tokenizer.eos_token
        
        print("✅ New Testament Expert model loaded successfully!")
    
    def ask_question(self, question, max_length=300):
        """
        Ask a New Testament related question
        """
        # Format the prompt specifically for New Testament queries
        prompt = f"""You are a New Testament expert. Please provide a detailed and accurate answer based on the New Testament scriptures.

Question: {question}
Answer:"""
        
        # Tokenize the input
        inputs = self.tokenizer.encode(prompt, return_tensors="pt")
        
        # Generate response
        with torch.no_grad():
            outputs = self.model.generate(
                inputs,
                max_length=max_length,
                num_return_sequences=1,
                temperature=0.7,
                do_sample=True,
                pad_token_id=self.tokenizer.eos_token_id
            )
        
        # Decode and clean the response
        response = self.tokenizer.decode(outputs[0], skip_special_tokens=True)
        answer = response.split("Answer:")[1].strip() if "Answer:" in response else response
        
        return answer
    
    def get_verse_explanation(self, verse_reference):
        """
        Get explanation for a specific New Testament verse
        """
        question = f"Please explain the meaning and context of {verse_reference} from the New Testament."
        return self.ask_question(question)
    
    def compare_gospels(self, topic):
        """
        Compare how different gospels present a topic
        """
        question = f"How do the four gospels (Matthew, Mark, Luke, and John) present {topic}? What are the similarities and differences?"
        return self.ask_question(question, max_length=400)

In [ ]:
# Initialize the New Testament Expert
nt_expert = NewTestamentExpert()

In [ ]:
# Test the New Testament Expert with various questions
questions = [
    "What are the Beatitudes and their significance?",
    "Who was Paul the Apostle and what were his main contributions?",
    "What is the Book of Revelation about?",
    "Explain the concept of salvation in the New Testament"
]

for question in questions:
    print(f"\n🔹 Question: {question}")
    answer = nt_expert.ask_question(question)
    print(f"📖 Answer: {answer}")
    print("-" * 80)

## Method 3: Verse Explanation and Gospel Comparison

In [ ]:
# Explain specific New Testament verses
verses = [
    "John 3:16",
    "Romans 8:28",
    "Philippians 4:13",
    "1 Corinthians 13:4-7"
]

print("📜 NEW TESTAMENT VERSE EXPLANATIONS")
print("=" * 50)

for verse in verses:
    print(f"\n📖 Verse: {verse}")
    explanation = nt_expert.get_verse_explanation(verse)
    print(f"💡 Explanation: {explanation}")
    print("-" * 50)

In [ ]:
# Compare how different gospels present various topics
topics = [
    "the birth of Jesus",
    "the crucifixion",
    "the resurrection"
]

print("⚖️ GOSPEL COMPARISONS")
print("=" * 40)

for topic in topics:
    print(f"\n🔍 Topic: {topic.title()}")
    comparison = nt_expert.compare_gospels(topic)
    print(f"📊 Comparison: {comparison}")
    print("-" * 60)

## Method 4: Interactive Q&A Session

In [ ]:
# Interactive session (uncomment to use)
# print("🔥 New Testament Expert - Interactive Session")
# print("Ask any question about the New Testament. Type 'exit' to quit.\n")

# while True:
#     user_question = input("❓ Your question: ")
#     
#     if user_question.lower() in ['exit', 'quit', 'bye']:
#         print("👋 Thank you for using the New Testament Expert!")
#         break
#     
#     if user_question.strip():
#         print("\n🤔 Thinking...")
#         answer = nt_expert.ask_question(user_question)
#         print(f"\n📖 Answer: {answer}\n")
#         print("-" * 80)

## Method 5: Using Different Biblical Models

In [ ]:
# Alternative biblical models available on Hugging Face
biblical_models = {
    "sleepdeprived3/Christian-Bible-Expert-v2.0-12B": "Primary biblical expert model",
    "sleepdeprived3/Reformed-Christian-Bible-Expert-12B_EXL2_5bpw_H8": "Reformed/Calvinist theological perspective",
    "aao331/ChristGPT-13B-V2-GPTQ": "Instruction-tuned model trained on bible to act like Jesus",
    "oliverbob/openbible": "Open biblical knowledge model",
    "abhi1nandy2/Bible-roberta-base": "RoBERTa model trained on biblical text",
    "Buttsac/bible": "Bible-focused language model",
}

print("📚 AVAILABLE BIBLICAL MODELS:")
print("=" * 60)
for model, description in biblical_models.items():
    print(f"🔹 {model}")
    print(f"   └─ {description}")
    print()

print("💡 For optimal New Testament expertise, consider:")
print("1. Fine-tuning a model on New Testament text")
print("2. Using retrieval-augmented generation (RAG) with New Testament corpus")
print("3. Implementing semantic search over New Testament verses")
print("4. Using the bible-nlp organization's datasets and tools")

## Method 6: Gradio Chat Interface

Interactive web-based chat interface for the New Testament Expert using Gradio.

In [ ]:
class GradioNewTestamentExpert:
    def __init__(self, model_name="sleepdeprived3/Christian-Bible-Expert-v2.0-12B"):
        self.model_name = model_name
        self.expert = None
        self.initialize_model()
    
    def initialize_model(self):
        """Initialize the model with proper error handling"""
        try:
            print(f"🔄 Loading {self.model_name}...")
            self.expert = NewTestamentExpert(self.model_name)
            print("✅ Model loaded successfully!")
        except Exception as e:
            print(f"❌ Error loading model: {e}")
            print("🔄 Falling back to pipeline approach...")
            self.expert = pipeline(
                "text-generation",
                model=self.model_name,
                torch_dtype=torch.float16,
                device_map="auto"
            )
    
    def chat_response(self, message, history):
        """Generate response for Gradio chat interface"""
        if not message.strip():
            return "Please ask a question about the New Testament."
        
        try:
            # Add context for New Testament focus
            nt_prompt = f"""You are a New Testament expert. Please provide a detailed and accurate answer based on the New Testament scriptures.

Question: {message}
Answer:"""
            
            if hasattr(self.expert, 'ask_question'):
                # Using the expert class
                response = self.expert.ask_question(message)
            else:
                # Using pipeline fallback
                result = self.expert(
                    nt_prompt,
                    max_length=300,
                    num_return_sequences=1,
                    temperature=0.7,
                    do_sample=True,
                    pad_token_id=self.expert.tokenizer.eos_token_id
                )
                response = result[0]['generated_text'].split('Answer:')[1].strip()
            
            return response
            
        except Exception as e:
            return f"Sorry, I encountered an error: {str(e)}. Please try rephrasing your question."
    
    def launch_chat(self, share=False):
        """Launch the Gradio chat interface"""
        
        # Custom CSS for better styling
        css = """
        .gradio-container {
            font-family: 'Arial', sans-serif;
        }
        .chat-message {
            padding: 10px;
            margin: 5px;
            border-radius: 10px;
        }
        """
        
        # Create the Gradio interface
        with gr.Blocks(css=css, title="New Testament Expert Chat") as demo:
            gr.Markdown(
                """
                # 📖 New Testament Expert Chat
                
                Welcome to the New Testament Expert! Ask any questions about:
                - New Testament books and verses
                - Jesus' teachings and parables
                - The apostles and early church
                - Biblical theology and interpretation
                - Gospel comparisons
                
                **Model**: `sleepdeprived3/Christian-Bible-Expert-v2.0-12B`
                """
            )
            
            chatbot = gr.Chatbot(
                [],
                elem_id="chatbot",
                height=400,
                bubble_full_width=False
            )
            
            with gr.Row():
                msg = gr.Textbox(
                    label="Your Question",
                    placeholder="Ask about the New Testament...",
                    container=False,
                    scale=7
                )
                submit_btn = gr.Button("Send", scale=1, variant="primary")
                clear_btn = gr.Button("Clear", scale=1)
            
            # Example questions
            gr.Examples(
                examples=[
                    "Who were the twelve apostles?",
                    "Explain the parable of the prodigal son",
                    "What is the significance of John 3:16?",
                    "Compare the resurrection accounts in the gospels",
                    "What are the Beatitudes?",
                    "Who was Paul and what were his main teachings?",
                    "Explain the concept of salvation in the New Testament"
                ],
                inputs=msg
            )
            
            def respond(message, chat_history):
                bot_message = self.chat_response(message, chat_history)
                chat_history.append((message, bot_message))
                return "", chat_history
            
            def clear_chat():
                return [], ""
            
            # Event handlers
            msg.submit(respond, [msg, chatbot], [msg, chatbot])
            submit_btn.click(respond, [msg, chatbot], [msg, chatbot])
            clear_btn.click(clear_chat, outputs=[chatbot, msg])
        
        print("🚀 Launching New Testament Expert Chat...")
        print("📱 The interface will open in your browser")
        
        return demo.launch(share=share, debug=True)

# Initialize the Gradio expert
gradio_expert = GradioNewTestamentExpert()

In [ ]:
# Launch the Gradio chat interface
# Set share=True to create a public link that can be shared
gradio_expert.launch_chat(share=False)

## Method 7: Multi-Model Comparison Chat

In [ ]:
class MultiModelBiblicalChat:
    def __init__(self):
        self.models = {
            "Christian Expert": "sleepdeprived3/Christian-Bible-Expert-v2.0-12B",
            "Reformed Expert": "sleepdeprived3/Reformed-Christian-Bible-Expert-12B_EXL2_5bpw_H8",
            "ChristGPT": "aao331/ChristGPT-13B-V2-GPTQ"
        }
        self.loaded_models = {}
    
    def load_model(self, model_name):
        """Load a specific model"""
        if model_name not in self.loaded_models:
            try:
                print(f"Loading {model_name}...")
                self.loaded_models[model_name] = pipeline(
                    "text-generation",
                    model=model_name,
                    torch_dtype=torch.float16,
                    device_map="auto",
                    trust_remote_code=True
                )
                print(f"✅ {model_name} loaded successfully!")
            except Exception as e:
                print(f"❌ Failed to load {model_name}: {e}")
                return None
        return self.loaded_models[model_name]
    
    def get_response(self, model_name, question):
        """Get response from a specific model"""
        model = self.load_model(model_name)
        if model is None:
            return f"Could not load {model_name}"
        
        try:
            prompt = f"""You are a New Testament expert. Please provide a detailed and accurate answer based on the New Testament scriptures.

Question: {question}
Answer:"""
            
            result = model(
                prompt,
                max_length=250,
                num_return_sequences=1,
                temperature=0.7,
                do_sample=True,
                pad_token_id=model.tokenizer.eos_token_id
            )
            
            response = result[0]['generated_text'].split('Answer:')[1].strip()
            return response
            
        except Exception as e:
            return f"Error generating response: {str(e)}"
    
    def create_comparison_interface(self):
        """Create Gradio interface for model comparison"""
        
        def compare_responses(question, selected_models):
            if not question.strip():
                return "Please enter a question."
            
            results = {}
            for model_type in selected_models:
                model_name = self.models[model_type]
                print(f"Getting response from {model_type}...")
                response = self.get_response(model_name, question)
                results[model_type] = response
            
            return results
        
        with gr.Blocks(title="Biblical Models Comparison") as demo:
            gr.Markdown(
                """
                # 🔍 Biblical Models Comparison
                
                Compare responses from different biblical AI models to the same New Testament question.
                Each model may provide different perspectives and insights.
                """
            )
            
            with gr.Row():
                question_input = gr.Textbox(
                    label="New Testament Question",
                    placeholder="Enter your biblical question here...",
                    lines=2
                )
            
            model_selection = gr.CheckboxGroup(
                choices=list(self.models.keys()),
                value=["Christian Expert"],
                label="Select Models to Compare"
            )
            
            compare_btn = gr.Button("Compare Responses 🔍", variant="primary")
            
            # Results display
            with gr.Row():
                for model_type in self.models.keys():
                    with gr.Column():
                        gr.Markdown(f"### {model_type}")
                        gr.Textbox(
                            label=f"{model_type} Response",
                            lines=8,
                            interactive=False,
                            elem_id=f"{model_type.lower().replace(' ', '_')}_output"
                        )
            
            # Example questions
            gr.Examples(
                examples=[
                    "What is the Great Commission?",
                    "Explain the parable of the talents",
                    "What does Paul teach about faith vs works?",
                    "Compare the birth narratives in Matthew and Luke"
                ],
                inputs=question_input
            )
            
            def update_outputs(question, models):
                results = compare_responses(question, models)
                outputs = []
                for model_type in self.models.keys():
                    if model_type in results:
                        outputs.append(results[model_type])
                    else:
                        outputs.append("")
                return outputs
            
            compare_btn.click(
                update_outputs,
                inputs=[question_input, model_selection],
                outputs=[gr.Textbox(elem_id=f"{model_type.lower().replace(' ', '_')}_output") 
                        for model_type in self.models.keys()]
            )
        
        return demo

# Initialize multi-model comparison
multi_model_chat = MultiModelBiblicalChat()

In [ ]:
# Launch the multi-model comparison interface
# comparison_demo = multi_model_chat.create_comparison_interface()
# comparison_demo.launch(share=False)

## Method 8: Advanced Gradio Features

In [ ]:
def create_advanced_nt_interface():
    """Advanced New Testament interface with multiple features"""
    
    # Initialize expert
    expert = NewTestamentExpert()
    
    def chat_with_context(message, history, context_type, temperature):
        """Enhanced chat with context selection"""
        contexts = {
            "General": "You are a New Testament expert. Provide accurate biblical answers.",
            "Pastoral": "You are a pastor providing pastoral guidance based on the New Testament.",
            "Academic": "You are a biblical scholar providing academic analysis of New Testament texts.",
            "Devotional": "You are providing devotional insights from the New Testament for spiritual growth."
        }
        
        context_prompt = contexts.get(context_type, contexts["General"])
        full_prompt = f"{context_prompt}\n\nQuestion: {message}\nAnswer:"
        
        try:
            # Use the expert's model directly with custom temperature
            inputs = expert.tokenizer.encode(full_prompt, return_tensors="pt")
            
            with torch.no_grad():
                outputs = expert.model.generate(
                    inputs,
                    max_length=350,
                    num_return_sequences=1,
                    temperature=temperature,
                    do_sample=True,
                    pad_token_id=expert.tokenizer.eos_token_id
                )
            
            response = expert.tokenizer.decode(outputs[0], skip_special_tokens=True)
            answer = response.split("Answer:")[1].strip() if "Answer:" in response else response
            
            return answer
            
        except Exception as e:
            return f"Error: {str(e)}"
    
    def verse_lookup(reference):
        """Look up and explain a Bible verse"""
        if not reference.strip():
            return "Please enter a verse reference (e.g., John 3:16)"
        
        return expert.get_verse_explanation(reference)
    
    def topic_study(topic, verses_to_include):
        """Create a topic study"""
        if not topic.strip():
            return "Please enter a topic to study"
        
        question = f"Provide a comprehensive study on {topic} in the New Testament"
        if verses_to_include:
            question += f", specifically including references to: {verses_to_include}"
        
        return expert.ask_question(question, max_length=500)
    
    # Create the interface
    with gr.Blocks(title="Advanced New Testament Expert", theme=gr.themes.Soft()) as demo:
        gr.Markdown(
            """
            # 📖 Advanced New Testament Expert
            
            Comprehensive New Testament study tool with multiple interaction modes.
            """
        )
        
        with gr.Tabs():
            # Chat Tab
            with gr.TabItem("💬 Chat"):
                with gr.Row():
                    with gr.Column(scale=3):
                        chatbot = gr.Chatbot(height=400)
                        msg = gr.Textbox(
                            label="Your Message",
                            placeholder="Ask about the New Testament..."
                        )
                        
                        with gr.Row():
                            send_btn = gr.Button("Send", variant="primary")
                            clear_btn = gr.Button("Clear")
                    
                    with gr.Column(scale=1):
                        context_type = gr.Radio(
                            choices=["General", "Pastoral", "Academic", "Devotional"],
                            value="General",
                            label="Response Style"
                        )
                        
                        temperature = gr.Slider(
                            minimum=0.1,
                            maximum=1.0,
                            value=0.7,
                            step=0.1,
                            label="Creativity Level"
                        )
                        
                        gr.Markdown(
                            """
                            **Response Styles:**
                            - **General**: Standard biblical answers
                            - **Pastoral**: Guidance and counseling focus
                            - **Academic**: Scholarly analysis
                            - **Devotional**: Spiritual growth focus
                            """
                        )
                
                def respond(message, chat_history, context, temp):
                    bot_message = chat_with_context(message, chat_history, context, temp)
                    chat_history.append((message, bot_message))
                    return "", chat_history
                
                msg.submit(respond, [msg, chatbot, context_type, temperature], [msg, chatbot])
                send_btn.click(respond, [msg, chatbot, context_type, temperature], [msg, chatbot])
                clear_btn.click(lambda: ([], ""), outputs=[chatbot, msg])
            
            # Verse Lookup Tab
            with gr.TabItem("🔍 Verse Lookup"):
                verse_input = gr.Textbox(
                    label="Verse Reference",
                    placeholder="e.g., John 3:16, Romans 8:28, Matthew 5:3-12"
                )
                verse_btn = gr.Button("Explain Verse", variant="primary")
                verse_output = gr.Textbox(
                    label="Explanation",
                    lines=10,
                    interactive=False
                )
                
                verse_btn.click(verse_lookup, inputs=verse_input, outputs=verse_output)
                
                gr.Examples(
                    examples=[
                        "John 3:16",
                        "Romans 8:28",
                        "Philippians 4:13",
                        "1 Corinthians 13:4-7",
                        "Ephesians 2:8-9",
                        "Matthew 28:19-20"
                    ],
                    inputs=verse_input
                )
            
            # Topic Study Tab
            with gr.TabItem("📚 Topic Study"):
                topic_input = gr.Textbox(
                    label="Topic",
                    placeholder="e.g., love, forgiveness, salvation, prayer"
                )
                verses_input = gr.Textbox(
                    label="Specific Verses to Include (optional)",
                    placeholder="e.g., John 3:16, Romans 3:23"
                )
                study_btn = gr.Button("Generate Study", variant="primary")
                study_output = gr.Textbox(
                    label="Topic Study",
                    lines=15,
                    interactive=False
                )
                
                study_btn.click(
                    topic_study,
                    inputs=[topic_input, verses_input],
                    outputs=study_output
                )
                
                gr.Examples(
                    examples=[
                        "Love in the New Testament",
                        "Forgiveness and redemption",
                        "The Holy Spirit's role",
                        "Prayer and meditation",
                        "Faith vs works",
                        "The kingdom of heaven"
                    ],
                    inputs=topic_input
                )
        
        gr.Markdown(
            """
            ---
            **Model**: `sleepdeprived3/Christian-Bible-Expert-v2.0-12B`
            
            **Tips for better results:**
            - Be specific in your questions
            - Include verse references when possible
            - Try different response styles for various perspectives
            """
        )
    
    return demo

# Create and launch the advanced interface
# advanced_demo = create_advanced_nt_interface()
# advanced_demo.launch(share=False)

## Summary

This notebook demonstrates multiple approaches to creating a New Testament expert using Hugging Face models:

### Core Methods:
1. **Pipeline Approach**: Simple text generation for quick questions
2. **Expert Class**: Structured approach with specialized methods
3. **Verse Explanation**: Detailed analysis of specific passages
4. **Gospel Comparison**: Comparative analysis across gospels
5. **Interactive Session**: Real-time Q&A functionality

### Gradio Chat Interfaces:
6. **Basic Gradio Chat**: Web-based chat interface with examples
7. **Multi-Model Comparison**: Compare responses from different biblical models
8. **Advanced Interface**: Tabbed interface with multiple features including:
   - Context-aware responses (General, Pastoral, Academic, Devotional)
   - Adjustable creativity/temperature settings
   - Dedicated verse lookup tool
   - Topic study generator

### Available Biblical Models:
- **sleepdeprived3/Christian-Bible-Expert-v2.0-12B**: Primary biblical expert model
- **sleepdeprived3/Reformed-Christian-Bible-Expert-12B_EXL2_5bpw_H8**: Reformed/Calvinist perspective
- **aao331/ChristGPT-13B-V2-GPTQ**: Model trained to act like Jesus
- **oliverbob/openbible**: Open biblical knowledge model
- **abhi1nandy2/Bible-roberta-base**: RoBERTa trained on biblical text
- **Buttsac/bible**: Bible-focused language model

### Key Features:
- Multiple response styles (pastoral, academic, devotional)
- Interactive web interfaces with Gradio
- Model comparison capabilities
- Verse-specific explanations
- Topic-based studies
- Temperature/creativity control
- Example questions and proper error handling

**Usage**: Uncomment the launch commands in the cells to run the interfaces. Each provides different levels of interaction and functionality for New Testament study and exploration.